In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

# Generate RSA key pair
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

# Save private key
with open("private.pem", "wb") as f:
    f.write(private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Save public key
with open("public.pem", "wb") as f:
    f.write(public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

print("Private & public keys generated!")


Private & public keys generated!


In [ ]:
from cryptography import x509
from cryptography.x509.oid import NameOID, ExtendedKeyUsageOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
import datetime

import tomllib as tl

with open("config.toml", "rb") as f:
    config = tl.load(f)['CA']

# Generate RSA Key Pair
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

# Create a Self-Signed X.509 Certificate
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, config['COMMON_NAME']),
    x509.NameAttribute(NameOID.COUNTRY_NAME, config['COUNTRY_NAME']),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, config['ORGANIZATION_NAME']),
    x509.NameAttribute(NameOID.ORGANIZATION_IDENTIFIER, config['ORGANIZATION_IDENTIFIER']),
    x509.NameAttribute(NameOID.SERIAL_NUMBER, config['SERIAL_NUMBER']),
])


cert = (x509.CertificateBuilder().subject_name(subject)
    .issuer_name(issuer)
    .public_key(public_key)
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.datetime.now(datetime.UTC))
    .not_valid_after(datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=365))
    .add_extension(
        x509.KeyUsage(
            digital_signature=True,
            content_commitment=True,  # Non-Repudiation
            key_encipherment=False,
            data_encipherment=False,
            key_agreement=False,
            key_cert_sign=False,
            crl_sign=False,
            encipher_only=False,
            decipher_only=False
        ), critical=True
    )
    .add_extension(
        x509.ExtendedKeyUsage([
            ExtendedKeyUsageOID.DOCUMENT_SIGNING
        ]), critical=True
    )
    .sign(private_key, hashes.SHA256())
)

# Save Private Key
with open("private.pem", "wb") as f:
    f.write(private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Save Public Certificate (X.509)
with open("certificate.pem", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

print("✅ X.509 Certificate & RSA Key Pair Generated!")


✅ X.509 Certificate & RSA Key Pair Generated!


In [3]:
import jwt
import json

# Load JSON invoice
with open("invoice.json", "r") as f:
    invoice_data = json.load(f)

# Load private key
with open("private.pem", "rb") as f:
    private_key = f.read()

# Create JWS signature
signed_invoice = jwt.encode(
    invoice_data,
    private_key,
    algorithm="RS256",
    headers={"alg": "RS256", "typ": "JWT"}
)

# Save signed JSON invoice
with open("signed_invoice.json", "w") as f:
    json.dump({"signed_invoice": signed_invoice}, f, indent=4)

print("Invoice successfully signed!")


Invoice successfully signed!


In [4]:
# Load public key
with open("public.pem", "rb") as f:
    public_key = f.read()

# Verify signature
decoded_invoice = jwt.decode(signed_invoice, public_key, algorithms=["RS256"])
print("Verified Invoice:", decoded_invoice)


Verified Invoice: {'invoice_id': 'INV-001', 'issue_date': '2025-03-21', 'supplier': {'name': 'My Company Sdn Bhd', 'id': '1234567890'}, 'customer': {'name': 'Customer Sdn Bhd', 'id': '0987654321'}, 'total_amount': 100.0, 'currency': 'MYR'}


In [12]:
import json
import base64
import jwt  # PyJWT
from cryptography.hazmat.primitives import serialization

def load_private_key():
    """Load private key from file."""
    with open("private.pem", "rb") as f:
        # return f.read()
        return serialization.load_pem_private_key(f.read(), password=None)

def preprocess_json_invoice(input_file, output_file):
    """Remove UBLExtensions & Signature, then minify JSON."""
    with open(input_file, "r") as f:
        invoice_data = json.load(f)
    
    # Remove unnecessary sections
    invoice_data.pop("UBLExtensions", None)
    invoice_data.pop("Signature", None)
    
    # Minify JSON (remove spaces/newlines)
    minified_json = json.dumps(invoice_data, separators=(",", ":"))
    
    with open(output_file, "w") as f:
        f.write(minified_json)
    
    return minified_json

def preprocess_dict_invoice(invoice_data):
    """Remove UBLExtensions & Signature from dict."""
    # Remove unnecessary sections
    invoice_data.pop("UBLExtensions", None)
    invoice_data.pop("Signature", None)
    return invoice_data

def sign_json_invoice(minified_json, private_key):
    """Sign JSON invoice using X.509 and JWS (RS256)."""
    return jwt.encode(
        json.loads(minified_json),  # Payload
        private_key,
        algorithm="RS256",
        headers={"alg": "RS256", "typ": "JWT"}
    )
    
def sign_dict_invoice(invoice_data, private_key):
    """Sign JSON invoice using X.509 and JWS (RS256)."""
    return jwt.encode(
        preprocess_dict_invoice(invoice_data),  # Payload
        private_key,
        algorithm="RS256",
        headers={"alg": "RS256", "typ": "JWT"}
    )

def encode_signed_invoice(signed_invoice):
    """Base64 encode the signed JSON invoice."""
    return base64.b64encode(signed_invoice.encode()).decode()

def main():
    private_key = load_private_key()
    minified_json = preprocess_json_invoice("invoice.json", "minified_invoice.json")
    signed_invoice = sign_json_invoice(minified_json, private_key)
    encoded_invoice = encode_signed_invoice(signed_invoice)
    
    with open("signed_invoice.json", "w") as f:
        json.dump({"signed_invoice": encoded_invoice}, f, indent=4)
    
    print("✔ Invoice signed and encoded successfully!")

if __name__ == "__main__":
    main()

✔ Invoice signed and encoded successfully!


generate X.509 certificate using cryptography

In [ ]:
sign = [{
    "UBLExtension": [{ 
        "ExtensionURI": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}], 
        "ExtensionContent": [{ 
            "UBLDocumentSignatures": [{ 
                "SignatureInformation": [{ 
                    "ID": [{"_": "urn:oasis:names:specification:ubl:signature:1"}], 
                    "ReferencedSignatureID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}], 
                    "Signature": [{
                        "Id": "signature", 
                        "Object": [{ 
                            "QualifyingProperties": [{ 
                                "Target": "signature", 
                                "SignedProperties": [{ 
                                    "Id": "id-xades-signed-props", 
                                    "SignedSignatureProperties": [{ 
                                        "SigningTime": [{"_": "<utcTimestamp>"}], 
                                        "SigningCertificate": [{ 
                                            "Cert": [{
                                                "CertDigest": [{ 
                                                    "DigestMethod": [{"_": "", "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"}], 
                                                    "DigestValue": [{"_": "<CertDigest>"}] 
                                                }],
                                                "IssuerSerial": [{ 
                                                    "X509IssuerName": [{"_": "<issuerName>"}], 
                                                    "X509SerialNumber": [{"_": "<CertSerialNumber>"}] 
                                                }] 
                                            }] 
                                        }] 
                                    }] 
                                }] 
                            }],
                        }], 
                        "KeyInfo": [{ 
                            "X509Data": [{ 
                                "X509Certificate": [{"_": "<x509Certificate>"}], 
                                "X509SubjectName": [{"_": "<subjectName>"}], 
                                "X509IssuerSerial": [{
                                    "X509IssuerName": [{"_": "<issuerName>"}], 
                                    "X509SerialNumber": [{"_": "<CertSerialNumber>"}] 
                                }] 
                            }] 
                        }], 
                        "SignatureValue": [{"_": "<Sig>"}], 
                        "SignedInfo": [{
                            "SignatureMethod": [{ 
                                "_": "",
                                "Algorithm": "http://www.w3.org/2001/04/xmldsig-more#rsa-sha256" 
                            }], 
                            "Reference": [{ 
                                "Type": "http://uri.etsi.org/01903/v1.3.2#SignedProperties", 
                                "URI": "#id-xades-signed-props", 
                                "DigestMethod": [{"_": "","Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"}], 
                                "DigestValue": [{"_": "<PropsDigest>"}] 
                            }, 
                            { 
                                "Type": "", 
                                "URI": "", 
                                "DigestMethod": [{"_": "","Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"}], 
                                "DigestValue": [{"_": "<DocDigest>"}] 
                            }]
                        }]
                    }]
                }]
            }]
        }] 
    }],
    "Signature": [{ 
        "ID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}], 
        "SignatureMethod": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}] 
    }] #end of “UBLExtensions” object  
} #end of “Invoice” objects key-value array 
]  #end of “Invoice” object  


In [14]:
sign = {
"UBLExtensions": [{
    "UBLExtension": [{
        "ExtensionURI": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}],
        "ExtensionContent": [{
            "UBLDocumentSignatures": [{
                "SignatureInformation": [{
                    "ID": [{"_": "urn:oasis:names:specification:ubl:signature:1"}],
                    "ReferencedSignatureID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}],
                    "Signature": [{
                        "Id": "signature",
                        "Object": [{
                            "QualifyingProperties": [{
                                "Target": "signature",
                                "SignedProperties": [{
                                    "Id": "id-xades-signed-props",
                                    "SignedSignatureProperties": [{
                                        # ! TODO: Assign UTC timestamp
                                        "SigningTime": [{"_": "2024-07-23T15:14:54Z"}], 
                                        "SigningCertificate": [{
                                            "Cert": [{
                                                "CertDigest": [{
                                                    "DigestMethod": [{
                                                        "_": "",
                                                        "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                                    }],
                                                    # ! TODO: Assign DigestValue
                                                    "DigestValue": [{"_": "KKBSTyiPKGkGl1AFqcPziKCEIDYGtnYUTQN4ukO7G40="}]
                                                }],
                                                "IssuerSerial": [{
                                                    # ! TODO: Assign Issuer Name and Serial Number
                                                    "X509IssuerName": [{"_": "CN=Trial LHDNM Sub CA V1, OU=Terms of use at http://www.posdigicert.com.my, O=LHDNM, C=MY"}],
                                                    "X509SerialNumber": [{"_": "162880276254639189035871514749820882117"}]
                                                }]
                                            }]
                                        }]
                                    }]
                                }]
                            }]
                        }],
                        "KeyInfo": [{
                            "X509Data": [{
                                # ! TODO: Assign X509Certificate, X509SubjectName, X509IssuerSerial
                                "X509Certificate": [{
                                    "_": "MIIFlDCCA3ygAwIBAgIQeomZorO+0AwmW2BRdWJMxTANBgkqhkiG9w0BAQsFADB1MQswCQYDVQQGEwJNWTEOMAwGA1UEChMFTEhETk0xNjA0BgNVBAsTLVRlcm1zIG9mIHVzZSBhdCBodHRwOi8vd3d3LnBvc2RpZ2ljZXJ0LmNvbS5teTEeMBwGA1UEAxMVVHJpYWwgTEhETk0gU3ViIENBIFYxMB4XDTI0MDYwNjAyNTIzNloXDTI0MDkwNjAyNTIzNlowgZwxCzAJBgNVBAYTAk1ZMQ4wDAYDVQQKEwVEdW1teTEVMBMGA1UEYRMMQzI5NzAyNjM1MDYwMRswGQYDVQQLExJUZXN0IFVuaXQgZUludm9pY2UxDjAMBgNVBAMTBUR1bW15MRIwEAYDVQQFEwlEMTIzNDU2NzgxJTAjBgkqhkiG9w0BCQEWFmFuYXMuYUBmZ3Zob2xkaW5ncy5jb20wggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQChvfOzAofnU60xFO7NcmF2WRi+dgor1D7ccISgRVfZC30Fdxnt1S6ZNf78Lbrz8TbWMicS8plh/pHy96OJvEBplsAgcZTd6WvaMUB2oInC86D3YShlthR6EzhwXgBmg/g9xprwlRqXMT2p4+K8zmyJZ9pIb8Y+tQNjm/uYNudtwGVm8A4hEhlRHbgfUXRzT19QZml6V2Ea0wQI8VyWWa8phCIkBD2w4F8jG4eP5/0XSQkTfBHHf+GV/YDJx5KiiYfmB1nGfwoPHix6Gey+wRjIq87on8Dm5+8ei8/bOhcuuSlpxgwphAP3rZrNbRN9LNVLSQ5md41asoBHfaDIVPVpAgMBAAGjgfcwgfQwHwYDVR0lBBgwFgYIKwYBBQUHAwQGCisGAQQBgjcKAwwwEQYDVR0OBAoECEDwms66hrpiMFMGA1UdIARMMEowSAYJKwYBBAGDikUBMDswOQYIKwYBBQUHAgEWLWh0dHBzOi8vd3d3LnBvc2RpZ2ljZXJ0LmNvbS5teS9yZXBvc2l0b3J5L2NwczATBgNVHSMEDDAKgAhNf9lrtsUI0DAOBgNVHQ8BAf8EBAMCBkAwRAYDVR0fBD0wOzA5oDegNYYzaHR0cDovL3RyaWFsY3JsLnBvc2RpZ2ljZXJ0LmNvbS5teS9UcmlhbExIRE5NVjEuY3JsMA0GCSqGSIb3DQEBCwUAA4ICAQBwptnIb1OA8NNVotgVIjOnpQtowew87Y0EBWAnVhOsMDlWXD/s+BL7vIEbX/BYa0TjakQ7qo4riSHyUkQ+X+pNsPEqolC4uFOp0pDsIdjsNB+WG15itnghkI99c6YZmbXcSFw9E160c7vG25gIL6zBPculHx5+laE59YkmDLdxx27e0TltUbFmuq3diYBOOf7NswFcDXCo+kXOwFfgmpbzYS0qfSoh3eZZtVHg0r6uga1UsMGb90+IRsk4st99EOVENvo0290lWhPBVK2G34+2TzbbYnVkoxnq6uDMw3cRpXX/oSfya+tyF51kT3iXvpmQ9OMF3wMlfKwCS7BZB2+iRja/1WHkAP7QW7/+0zRBcGQzY7AYsdZUllwYapsLEtbZBrTiH12X4XnZjny9rLfQLzJsFGT7Q+e02GiCsBrK7ZHNTindLRnJYAo4U2at5+SjqBiXSmz0DG+juOyFkwiWyD0xeheg4tMMO2pZ7clQzKflYnvFTEFnt+d+tvVwNjTboxfVxEv2qWF6qcMJeMvXwKTXuwVI2iUqmJSzJbUY+w3OeG7fvrhUfMJPM9XZBOp7CEI1QHfHrtyjlKNhYzG3IgHcfAZUURO16gFmWgzAZLkJSmCIxaIty/EmvG5N3ZePolBOa7lNEH/eSBMGAQteH+Twtiu0Y2xSwmmsxnfJyw=="
                                }],
                                # ! TODO: Assign X509SubjectName
                                "X509SubjectName": [{
                                    "_": "CN=Trial LHDNM Sub CA V1, OU=Terms of use at http://www.posdigicert.com.my, O=LHDNM, C=MY"
                                }],
                                # ! TODO: Assign X509IssuerSerial, it's the same as the IssuerSerial in CertDigest
                                "X509IssuerSerial": [{
                                    "X509IssuerName": [{"_": "CN=Trial LHDNM Sub CA V1, OU=Terms of use at http://www.posdigicert.com.my, O=LHDNM, C=MY"}],
                                    "X509SerialNumber": [{
                                        "_": "162880276254639189035871514749820882117"
                                    }]
                                }]
                            }]
                        }],
                        # ! TODO: Assign SignatureValue
                        "SignatureValue": [{"_": "QTvntg4opuS7ZYWmly/iAO2OnLVJcKylYuF+QJKZdx9BkFVglmVuFtEtwoqgNsbsKaaEDinTSUAVStRJs2tiU1Jdryd4hoZ/Hc5TAvFnThpauVOLsc3j07cUB1+zhNjENmFeI9yzTGjr8XfNi4mNPspnhFAT4QGbRpxkWiIsKj762p3dhCwUNAuNLjunVaosYQ5lvSzGt4B9TF/1xJ7Z6kdcJTmBeltTWErSRA2EOMzWsGWGZVvyPLnXfnlIBQItTvARXveafxFdS1iw91g7mSEEYeqEviI0b4FUmkwH8ed0boFc6EHl1VF+2uVxBtHeKf31FqTQl/6/pF4Qgpn6Hg=="}],
                        "SignedInfo": [{
                            "SignatureMethod": [{
                                "_": "",
                                "Algorithm": "http://www.w3.org/2001/04/xmldsig-more#rsa-sha256"
                            }],
                            "Reference": [{
                                "Type": "http://uri.etsi.org/01903/v1.3.2#SignedProperties",
                                "URI": "#id-xades-signed-props",
                                "DigestMethod": [{
                                    "_": "",
                                    "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                }],
                                "DigestValue": [{
                                    "_": "Rzuzz+70GSnGBF1YxhHnjSzFpQ1MW4vyX/Q9bTHkE2c="
                                }]
                            },
                            {
                                "Type": "",
                                "URI": "",
                                "DigestMethod": [{
                                    "_": "",
                                    "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                }],
                                "DigestValue": [{
                                    "_": "vMs/IdnS7isftqrBDr4F1LK/CkvBkW5Gb3Wn6OVzAxo="
                                }]
                            }]
                        }]
                    }]
                }]
            }]
        }]
    }]
}],
"Signature": [{
    "ID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}],
    "SignatureMethod": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}]
}]
}

import hashlib
import base64

def digest_value(cert: str) -> str:
    '''
    # Convert the certificate to a SHA-256 digest value.
    
    **Steps:**
    1. Remove the header and footer of the certificate
    2. Remove newlines
    3. Calculate SHA-256 hash
    4. Base64 encode the hash
    '''
    cert = cert.replace("-----BEGIN CERTIFICATE-----", "").replace("-----END CERTIFICATE-----", "").replace("\n", "")
    
    sha256 = hashlib.sha256()
    sha256.update(cert.encode())
    return base64.b64encode(sha256.digest()).decode()

from cryptography import x509

from typing import Tuple

def issuer_serial(cert: str) -> Tuple[str, int]:
    '''
    # Retrieve the issuer name and serial number
    '''
    cert_ = x509.load_pem_x509_certificate(cert.encode())
    
    return cert_.issuer.rfc4514_string(), cert_.serial_number

def subject_name(cert: str) -> str:
    '''
    # Retrieve the subject name
    '''
    cert_ = x509.load_pem_x509_certificate(cert.encode())
    
    return cert_.subject.rfc4514_string()

from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes, serialization

def sign_data_(json_data: str, cert: str, private: str) -> str:
    '''
    # Signs the given JSON data using the provided private key and returns the Base64-encoded signature.
    
    **Steps:**
    1. Minify the JSON data
    2. Compute the SHA-256 hash of the minified JSON
    3. Load the private key
    4. Sign the hash
    5. Convert the signature to Base
    '''
    # Load and minify JSON data
    minified_json = json.dumps(json.loads(json_data), separators=(",", ":"))
    
    # Compute SHA-256 hash of the minified JSON
    hash_obj = hashlib.sha256(minified_json.encode('utf-8')).digest()
    
    # Load private key
    with open(private, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None
        )
    
    # Sign the hash
    signature = private_key.sign(
        hash_obj,
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    
    # Convert signature to Base64
    signature_base64 = base64.b64encode(signature).decode('utf-8')
    
    return signature_base64

import json
import jwt

def prop_digest(
    SigningTime: str,
    certDigest: str,
    issuerName: str,
    CertSerialNumber: str
) -> str:
    digest_val = {
        "Target":"signature",
        "SignedProperties":[{
            "Id":"id-xades-signed props",
            "SignedSignatureProperties":[{
                "SigningTime":[{"_":SigningTime}],
                "SigningCertificate":[{
                    "Cert":[{
                        "CertDigest":[{
                            "DigestMethod":[{"_":"","Algorithm":"http://www.w3.org/2001/04/xmlenc#sha256"}],
                            "DigestValue":[{"_":certDigest}]}],
                        "IssuerSerial":[{
                            "X509IssuerName":[{"_":issuerName}],
                            "X509SerialNumber":[{"_":CertSerialNumber}]
                        }]
                    }]
                }]
            }]
        }]
    }
    
    # to minified json string
    signedprops = json.dumps(digest_val, separators=(",", ":"))
    signedpropshash = hashlib.sha256(signedprops.encode()).digest()
    return base64.b64encode(signedpropshash).decode()

# * Calculate digest of json doc, without the ublextensions and signature
def doc_digest(json_data: str) -> Tuple[str, str]:
    '''
    # Calculate the SHA-256 digest of the JSON document.
    '''
    # Load and minify JSON data
    minified_json = json.dumps(json.loads(json_data), separators=(",", ":"))
    
    # Compute SHA-256 hash of the minified JSON
    hash_obj = hashlib.sha256(minified_json.encode('utf-8')).digest()
    
    # return hash and base64 encoded hash
    return (
        hash_obj,
        base64.b64encode(hash_obj).decode('utf-8')
    )

# * Sign the document using the hash
def sign_data(hash_obj: dict, private) -> str:
    '''
    # Signs the given JSON data using the provided private key and returns the Base64-encoded signature.
    
    **Steps:**
    1. Load the private key
    2. Sign the hash
    3. Convert the signature to Base64
    '''
    # # Load private key
    # with open(private, "rb") as key_file:
    #     private_key = serialization.load_pem_private_key(
    #         key_file.read(),
    #         password=None
    #     )
    
    # Convert JSON object to string and encode to bytes
    json_bytes = json.dumps(hash_obj, separators=(",", ":")).encode('utf-8')
    
    # Hash the JSON data
    hash_obj = hashes.Hash(hashes.SHA256())
    hash_obj.update(json_bytes)
    hashed_data = hash_obj.finalize()
    
    # Sign the hash
    signature = private.sign(
        hashed_data,
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    
    # Convert signature to Base64
    signature_base64 = base64.b64encode(signature).decode('utf-8')
    
    return signature_base64

# * Calculate the certificate digest
def cert_digest(cert: str) -> str:
    '''
    # Calculate the SHA-256 digest of the certificate.
    '''
    # Load certificate
    cert_ = x509.load_pem_x509_certificate(cert.encode())
    
    # Compute SHA-256 hash of the certificate
    hash_obj = hashlib.sha256(cert_.public_bytes(encoding=serialization.Encoding.PEM)).digest()
    
    # return base64 encoded hash
    return base64.b64encode(hash_obj).decode('utf-8')


def get_sign(
    timestamp: str,
    cert_digest: str, # in base64
    issuer_name: str,
    cert_serial: str,
    x509_cert: str, # in base64
    x509_subject: str,
    signature: str, # in base64
    props_digest: str, # in base64
    doc_digest: str, # in base64
):
    ...
    sign = {
    "UBLExtensions": [{
        "UBLExtension": [{
            "ExtensionURI": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}],
            "ExtensionContent": [{
                "UBLDocumentSignatures": [{
                    "SignatureInformation": [{
                        "ID": [{"_": "urn:oasis:names:specification:ubl:signature:1"}],
                        "ReferencedSignatureID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}],
                        "Signature": [{
                            "Id": "signature",
                            "Object": [{
                                "QualifyingProperties": [{
                                    "Target": "signature",
                                    "SignedProperties": [{
                                        "Id": "id-xades-signed-props",
                                        "SignedSignatureProperties": [{
                                            # ! TODO: Assign UTC timestamp
                                            "SigningTime": [{"_": "2024-07-23T15:14:54Z"}], 
                                            "SigningCertificate": [{
                                                "Cert": [{
                                                    "CertDigest": [{
                                                        "DigestMethod": [{
                                                            "_": "",
                                                            "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                                        }],
                                                        # ! TODO: Assign DigestValue
                                                        "DigestValue": [{"_": "KKBSTyiPKGkGl1AFqcPziKCEIDYGtnYUTQN4ukO7G40="}]
                                                    }],
                                                    "IssuerSerial": [{
                                                        # ! TODO: Assign Issuer Name and Serial Number
                                                        "X509IssuerName": [{"_": "CN=Trial LHDNM Sub CA V1, OU=Terms of use at http://www.posdigicert.com.my, O=LHDNM, C=MY"}],
                                                        "X509SerialNumber": [{"_": "162880276254639189035871514749820882117"}]
                                                    }]
                                                }]
                                            }]
                                        }]
                                    }]
                                }]
                            }],
                            "KeyInfo": [{
                                "X509Data": [{
                                    # ! TODO: Assign X509Certificate, X509SubjectName, X509IssuerSerial
                                    "X509Certificate": [{
                                        "_": "MIIFlDCCA3ygAwIBAgIQeomZorO+0AwmW2BRdWJMxTANBgkqhkiG9w0BAQsFADB1MQswCQYDVQQGEwJNWTEOMAwGA1UEChMFTEhETk0xNjA0BgNVBAsTLVRlcm1zIG9mIHVzZSBhdCBodHRwOi8vd3d3LnBvc2RpZ2ljZXJ0LmNvbS5teTEeMBwGA1UEAxMVVHJpYWwgTEhETk0gU3ViIENBIFYxMB4XDTI0MDYwNjAyNTIzNloXDTI0MDkwNjAyNTIzNlowgZwxCzAJBgNVBAYTAk1ZMQ4wDAYDVQQKEwVEdW1teTEVMBMGA1UEYRMMQzI5NzAyNjM1MDYwMRswGQYDVQQLExJUZXN0IFVuaXQgZUludm9pY2UxDjAMBgNVBAMTBUR1bW15MRIwEAYDVQQFEwlEMTIzNDU2NzgxJTAjBgkqhkiG9w0BCQEWFmFuYXMuYUBmZ3Zob2xkaW5ncy5jb20wggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQChvfOzAofnU60xFO7NcmF2WRi+dgor1D7ccISgRVfZC30Fdxnt1S6ZNf78Lbrz8TbWMicS8plh/pHy96OJvEBplsAgcZTd6WvaMUB2oInC86D3YShlthR6EzhwXgBmg/g9xprwlRqXMT2p4+K8zmyJZ9pIb8Y+tQNjm/uYNudtwGVm8A4hEhlRHbgfUXRzT19QZml6V2Ea0wQI8VyWWa8phCIkBD2w4F8jG4eP5/0XSQkTfBHHf+GV/YDJx5KiiYfmB1nGfwoPHix6Gey+wRjIq87on8Dm5+8ei8/bOhcuuSlpxgwphAP3rZrNbRN9LNVLSQ5md41asoBHfaDIVPVpAgMBAAGjgfcwgfQwHwYDVR0lBBgwFgYIKwYBBQUHAwQGCisGAQQBgjcKAwwwEQYDVR0OBAoECEDwms66hrpiMFMGA1UdIARMMEowSAYJKwYBBAGDikUBMDswOQYIKwYBBQUHAgEWLWh0dHBzOi8vd3d3LnBvc2RpZ2ljZXJ0LmNvbS5teS9yZXBvc2l0b3J5L2NwczATBgNVHSMEDDAKgAhNf9lrtsUI0DAOBgNVHQ8BAf8EBAMCBkAwRAYDVR0fBD0wOzA5oDegNYYzaHR0cDovL3RyaWFsY3JsLnBvc2RpZ2ljZXJ0LmNvbS5teS9UcmlhbExIRE5NVjEuY3JsMA0GCSqGSIb3DQEBCwUAA4ICAQBwptnIb1OA8NNVotgVIjOnpQtowew87Y0EBWAnVhOsMDlWXD/s+BL7vIEbX/BYa0TjakQ7qo4riSHyUkQ+X+pNsPEqolC4uFOp0pDsIdjsNB+WG15itnghkI99c6YZmbXcSFw9E160c7vG25gIL6zBPculHx5+laE59YkmDLdxx27e0TltUbFmuq3diYBOOf7NswFcDXCo+kXOwFfgmpbzYS0qfSoh3eZZtVHg0r6uga1UsMGb90+IRsk4st99EOVENvo0290lWhPBVK2G34+2TzbbYnVkoxnq6uDMw3cRpXX/oSfya+tyF51kT3iXvpmQ9OMF3wMlfKwCS7BZB2+iRja/1WHkAP7QW7/+0zRBcGQzY7AYsdZUllwYapsLEtbZBrTiH12X4XnZjny9rLfQLzJsFGT7Q+e02GiCsBrK7ZHNTindLRnJYAo4U2at5+SjqBiXSmz0DG+juOyFkwiWyD0xeheg4tMMO2pZ7clQzKflYnvFTEFnt+d+tvVwNjTboxfVxEv2qWF6qcMJeMvXwKTXuwVI2iUqmJSzJbUY+w3OeG7fvrhUfMJPM9XZBOp7CEI1QHfHrtyjlKNhYzG3IgHcfAZUURO16gFmWgzAZLkJSmCIxaIty/EmvG5N3ZePolBOa7lNEH/eSBMGAQteH+Twtiu0Y2xSwmmsxnfJyw=="
                                    }],
                                    # ! TODO: Assign X509SubjectName
                                    "X509SubjectName": [{
                                        "_": "CN=Trial LHDNM Sub CA V1, OU=Terms of use at http://www.posdigicert.com.my, O=LHDNM, C=MY"
                                    }],
                                    # ! TODO: Assign X509IssuerSerial, it's the same as the IssuerSerial in CertDigest
                                    "X509IssuerSerial": [{
                                        "X509IssuerName": [{"_": "CN=Trial LHDNM Sub CA V1, OU=Terms of use at http://www.posdigicert.com.my, O=LHDNM, C=MY"}],
                                        "X509SerialNumber": [{
                                            "_": "162880276254639189035871514749820882117"
                                        }]
                                    }]
                                }]
                            }],
                            # ! TODO: Assign SignatureValue
                            "SignatureValue": [{"_": "QTvntg4opuS7ZYWmly/iAO2OnLVJcKylYuF+QJKZdx9BkFVglmVuFtEtwoqgNsbsKaaEDinTSUAVStRJs2tiU1Jdryd4hoZ/Hc5TAvFnThpauVOLsc3j07cUB1+zhNjENmFeI9yzTGjr8XfNi4mNPspnhFAT4QGbRpxkWiIsKj762p3dhCwUNAuNLjunVaosYQ5lvSzGt4B9TF/1xJ7Z6kdcJTmBeltTWErSRA2EOMzWsGWGZVvyPLnXfnlIBQItTvARXveafxFdS1iw91g7mSEEYeqEviI0b4FUmkwH8ed0boFc6EHl1VF+2uVxBtHeKf31FqTQl/6/pF4Qgpn6Hg=="}],
                            "SignedInfo": [{
                                "SignatureMethod": [{
                                    "_": "",
                                    "Algorithm": "http://www.w3.org/2001/04/xmldsig-more#rsa-sha256"
                                }],
                                "Reference": [{
                                    "Type": "http://uri.etsi.org/01903/v1.3.2#SignedProperties",
                                    "URI": "#id-xades-signed-props",
                                    "DigestMethod": [{
                                        "_": "",
                                        "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                    }],
                                    "DigestValue": [{
                                        # ! TODO: Assign PropsDigest
                                        "_": "Rzuzz+70GSnGBF1YxhHnjSzFpQ1MW4vyX/Q9bTHkE2c="
                                    }]
                                },
                                {
                                    "Type": "",
                                    "URI": "",
                                    "DigestMethod": [{
                                        "_": "",
                                        "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                    }],
                                    "DigestValue": [{
                                        # ! TODO: Assign DocDigest
                                        "_": "vMs/IdnS7isftqrBDr4F1LK/CkvBkW5Gb3Wn6OVzAxo="
                                    }]
                                }]
                            }]
                        }]
                    }]
                }]
            }]
        }]
    }],
    "Signature": [{
        "ID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}],
        "SignatureMethod": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}]
    }]
    }
    
    return sign




In [ ]:
# generate signature dict invoice.json using method above
import json
import jwt
import base64
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPrivateKey
from cryptography.hazmat.primitives.asymmetric import padding
import datetime

with open("invoice.json", "r") as f:
    invoice_data = json.load(f)
    
with open("private.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None
    )

with open("certificate.pem", "r") as f:
    cert = f.read()

timestamp: str = datetime.datetime.now(datetime.timezone.utc).isoformat()
cert_d: str = cert_digest(cert)
issuer_serial_ = issuer_serial(cert)
issuer_name: str = issuer_serial_[0]
cert_serial: str = issuer_serial_[1]
x509_cert: str = base64.b64encode(cert.encode()).decode()
x509_subject: str = subject_name(cert)
signature: str = sign_data(invoice_data, private_key)
props_digest: str = prop_digest(timestamp, cert_d, issuer_name, cert_serial)
doc_digest_: str = doc_digest(json.dumps(invoice_data))[1]

get_sign(
    timestamp,
    cert_d,
    issuer_name,
    cert_serial,
    x509_cert,
    x509_subject,
    signature,
    props_digest,
    doc_digest_
)


In [ ]:
import json
import base64
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPrivateKey
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography import x509
from cryptography.x509 import Certificate
import datetime


class SignService:
    def __init__(
        self,
        invoice_data: dict,
        private_key: RSAPrivateKey,
        cert: Certificate
    ):
        self.invoice_data = invoice_data
        self.invoice_data_minified = json.dumps(invoice_data, separators=(",", ":"))
        self.private_key = private_key
        self.cert = cert
        
    def digest_doc(self) -> str:
        hash_obj = hashlib.sha256(self.invoice_data_minified.encode('utf-8')).digest()
        return base64.b64encode(hash_obj).decode('utf-8')
    
    def sign_doc(self) -> str:
        hash_obj = hashlib.sha256(self.invoice_data_minified.encode('utf-8')).digest()
        signature = self.private_key.sign(
            hash_obj,
            padding.PKCS1v15(),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    
    def digest_cert(self) -> str:
        hash_obj = hashlib.sha256(self.cert.public_bytes(encoding=serialization.Encoding.PEM)).digest()
        return base64.b64encode(hash_obj).decode('utf-8')

    def gen_sign_props(self) -> dict:
        self.sign_props = {
            "Target":"signature",
            "SignedProperties":[{
                "Id":"id-xades-signed props",
                "SignedSignatureProperties":[{
                    "SigningTime":[{"_":datetime.datetime.now(datetime.timezone.utc).isoformat()}],
                    "SigningCertificate":[{
                        "Cert":[{
                            "CertDigest":[{
                                "DigestMethod":[{"_":"","Algorithm":"http://www.w3.org/2001/04/xmlenc#sha256"}],
                                "DigestValue":[{"_":self.digest_cert()}]}],
                            "IssuerSerial":[{
                                "X509IssuerName":[{"_":self.cert.issuer.rfc4514_string()}],
                                "X509SerialNumber":[{"_":self.cert.serial_number}]
                            }]
                        }]
                    }]
                }]
            }]
        }
        return self.sign_props
    
    def digest_sign_props(self) -> str:
        signedprops = json.dumps(self.sign_props, separators=(",", ":"))
        signedpropshash = hashlib.sha256(signedprops.encode()).digest()
        return base64.b64encode(signedpropshash).decode()
    
    def gen_sign(self) -> dict:
        self.gen_sign_props()
        self.sign = {
            "UBLExtensions": [{
                "UBLExtension": [{
                    "ExtensionURI": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}],
                    "ExtensionContent": [{
                        "UBLDocumentSignatures": [{
                            "SignatureInformation": [{
                                "ID": [{"_": "urn:oasis:names:specification:ubl:signature:1"}],
                                "ReferencedSignatureID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}],
                                "Signature": [{
                                    "Id": "signature",
                                    "Object": [{
                                        "QualifyingProperties": [{
                                            "Target": "signature",
                                            "SignedProperties": [{
                                                "Id": "id-xades-signed-props",
                                                "SignedSignatureProperties": [{
                                                    # ! TODO: Assign UTC timestamp
                                                    "SigningTime": [{"_": datetime.datetime.now(datetime.timezone.utc).isoformat()}],
                                                    "SigningCertificate": [{
                                                        "Cert": [{
                                                            "CertDigest": [{
                                                                "DigestMethod": [{
                                                                    "_": "",
                                                                    "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                                                }],
                                                                # ! TODO: Assign DigestValue
                                                                "DigestValue": [{"_": self.digest_cert()}]
                                                            }],
                                                            "IssuerSerial": [{
                                                                # ! TODO: Assign Issuer Name and Serial Number
                                                                "X509IssuerName": [{"_": self.cert.issuer.rfc4514_string()}],
                                                                "X509SerialNumber": [{"_": self.cert.serial_number}]
                                                            }]
                                                        }]
                                                    }]
                                                }]
                                            }]
                                        }]
                                    }],
                                    "KeyInfo": [{
                                        "X509Data": [{
                                            # ! TODO: Assign X509Certificate, X509SubjectName, X509IssuerSerial
                                            "X509Certificate": [{
                                                "_": base64.b64encode(self.cert.public_bytes(encoding=serialization.Encoding.PEM)).decode()
                                            }],
                                            # ! TODO: Assign X509SubjectName
                                            "X509SubjectName": [{
                                                "_": self.cert.subject.rfc4514_string()
                                            }],
                                            # ! TODO: Assign X509IssuerSerial
                                            "X509IssuerSerial": [{
                                                "X509IssuerName": [{"_": self.cert.issuer.rfc4514_string()}],
                                                "X509SerialNumber": [{
                                                    "_": self.cert.serial_number
                                                }]
                                            }]
                                        }]
                                    }],
                                    # ! TODO: Assign SignatureValue
                                    "SignatureValue": [{"_": self.sign_doc()}],
                                    "SignedInfo": [{
                                        "SignatureMethod": [{
                                            "_": "",
                                            "Algorithm": "http://www.w3.org/2001/04/xmldsig-more#rsa-sha256"
                                        }],
                                        "Reference": [{
                                            "Type": "http://uri.etsi.org/01903/v1.3.2#SignedProperties",
                                            "URI": "#id-xades-signed-props",
                                            "DigestMethod": [{
                                                "_": "",
                                                "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                            }],
                                            "DigestValue": [{
                                                # ! TODO: Assign PropsDigest
                                                "_": self.digest_sign_props()
                                            }]
                                        },
                                        {
                                            "Type": "",
                                            "URI": "",
                                            "DigestMethod": [{
                                                "_": "",
                                                "Algorithm": "http://www.w3.org/2001/04/xmlenc#sha256"
                                            }],
                                            "DigestValue": [{
                                                # ! TODO: Assign DocDigest
                                                "_": self.digest_doc()
                                            }]
                                        }]
                                    }]
                                }]
                            }]
                        }]
                    }]
                }]
            }],
            "Signature": [{
                "ID": [{"_": "urn:oasis:names:specification:ubl:signature:Invoice"}],
                "SignatureMethod": [{"_": "urn:oasis:names:specification:ubl:dsig:enveloped:xades"}]
            }]
        }
        
        return self.sign


with open("invoice.json", "r") as f:
    invoice_data = json.load(f)
    
with open("private.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None
    )

with open("certificate.pem", "r") as f:
    cert_str = f.read()
    cert = x509.load_pem_x509_certificate(cert_str.encode())
    
sign_service = SignService(invoice_data, private_key, cert)
sign_service.gen_sign()

